In [1]:
import pandas as pd
import numpy as np
import bokeh.plotting as bpl
import bokeh.layouts as bly
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_log_error, mean_squared_error

from pathlib import Path
from herramientas import arbol
from datetime import datetime as dt

In [2]:
DATOS_BRUTOS = Path("../datos/brutos")
DATOS_EXTERNOS = Path("../datos/externos")

In [3]:
bpl.output_notebook()

Loading BokehJS ...

Definición del modelo de CDMX, los detalles del modelo se encuentran en [https://modelo.covid19.cdmx.gob.mx/modelo-epidemico](https://modelo.covid19.cdmx.gob.mx/modelo-epidemico)

In [4]:
# Susceptibles equation
def dS_dt(S, I, R_t, t_inf):
    return -(R_t / t_inf) * I * S

# Expuestos equation
def dE_dt(S, E, I, R_t, t_inf, t_inc):
    return (R_t / t_inf) * I * S - (E / t_inc)

# Infectados equation
def dI_dt(I, E, t_inc, t_inf):
    return (E / t_inc) - (I / t_inf)

#Leves equation
def dL_dt(I, L, p_grave, t_inf, t_rl):
    return (1-p_grave)*(I /t_inf) - (L / t_rl)

#Graves equation
def dG_dt(I, G, p_grave, t_inf, t_hosp):
    return p_grave*(I / t_inf) - (G / t_hosp)

# Hospializados equation
def dH_dt(G, H, t_hosp, p_icu, t_rh, t_icu):
    return (G / t_hosp) - (1-p_icu)*(H / t_rh) - (p_icu)*(H / t_icu)

# ICU equation
def dICU_dt(H, ICU, p_icu, t_icu, p_m, t_ricu, t_m):
    return p_icu*(H / t_icu) - (1-p_m)*(ICU / t_ricu) - p_m*(ICU / t_m)

# Recovered equation
def dR_dt(L, H, ICU, t_rl, p_icu, t_rh, p_m, t_ricu):
    return (L / t_rl) + (1-p_icu)*(H / t_rh) + (1-p_m)*(ICU / t_ricu)

# Deaths equation
def dD_dt(ICU, t_m, p_m):
    return p_m*(ICU / t_m)


def CDMX_model(t, y, R_t, t_inf=2.9, t_inc=5.2, t_rh=12, t_rl=14, t_hosp=4, t_icu=1, t_ricu=7, t_m=8, p_m=.03, p_grave=.138, p_icu=0.05):

    if callable(R_t):
        reprod = R_t(t)
    else:
        reprod = R_t
        
    S, E, I, L, G, H, ICU, R, D = y
    
    S_out = dS_dt(S, I, reprod, t_inf)
    E_out = dE_dt(S, E, I, reprod, t_inf, t_inc)
    I_out = dI_dt(I, E, t_inc, t_inf)
    L_out = dL_dt(I, L, p_grave, t_inf, t_rl)
    G_out = dG_dt(I, G, p_grave, t_inf, t_hosp)
    H_out = dH_dt(G, H, t_hosp, p_icu, t_rh, t_icu)
    ICU_out = dICU_dt(H, ICU, p_icu, t_icu, p_m, t_ricu, t_m)
    R_out = dR_dt(L, H, ICU, t_rl, p_icu, t_rh, p_m, t_ricu)
    D_out = dD_dt(ICU, t_m, p_m)
    return [S_out, E_out, I_out, L_out, G_out, H_out, ICU_out, R_out, D_out]

Ejecución del modelo de CDMX con los parámetros que reportan, hay dos instancias, una sin intervenciones y la otra con 3 intervenciones

In [5]:
N = 22000000
n_infected = 556*2
max_days = 200
inicio = '2020-03-19'
tiempos = pd.date_range(start=inicio, periods=max_days).values
serie_tiempos = pd.Series(tiempos)

initial_state = [(N - n_infected)/ N, 0, n_infected / N, 0, 0, 0, 0, 0, 0]

R_0 = 3.8
t_inf=2.9
t_inc=5.2
t_rh=12
t_rl=14
t_hosp=4
t_icu=1
t_ricu = 7
t_m=8
p_m=0.65
p_grave=0.138
p_icu=0.05

intervenciones = [{"R":2.83,"fecha":inicio},
                  {"R":2.2,"fecha":"2020-03-22"},
                  {"R":0.95,"fecha":"2020-04-01"},
                  {"R":1.6,"fecha":"2020-04-13"},
                  {"R":0.95,"fecha":"2020-04-20"}]

def rep(t):
    rfinal = R_0
    for intervencion in intervenciones:
        tint = serie_tiempos[(serie_tiempos-pd.Timestamp(intervencion["fecha"])).dt.days==0].index[0]
        if (t > tint):
            rfinal = intervencion["R"]
    return rfinal

args_sin_interv = (R_0, t_inf, t_inc, t_rh, t_rl, t_hosp, t_icu, t_ricu, t_m, p_m, p_grave, p_icu)
args_con_interv = (rep, t_inf, t_inc, t_rh, t_rl, t_hosp, t_icu, t_ricu, t_m, p_m, p_grave, p_icu)

sol_sin_interv = solve_ivp(CDMX_model, [0, max_days], initial_state, args=args_sin_interv, t_eval=np.arange(max_days))
sol_con_interv = solve_ivp(CDMX_model, [0, max_days], initial_state, args=args_con_interv, t_eval=np.arange(max_days))

In [6]:
labels = ["Susceptibles","Expuestos","Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]
colores = {"Susceptibles":"#969696",
           "Recuperados":"Green",
           "Expuestos":"Pink",
           "Infectados":"Red",
           "Leves":"#807DBA",
           "Graves":"#54278F",
           "Hospitalizados":"#CC4C02",
           "ICUs":"#662506",
           "Defunciones":"#525252"}

In [7]:
solucion_sin = pd.DataFrame(sol_sin_interv.y.T*N,columns=labels,index=sol_sin_interv.t)
solucion_con = pd.DataFrame(sol_con_interv.y.T*N,columns=labels,index=sol_con_interv.t)

In [8]:
solucion_sin["Fecha"] = tiempos
solucion_con["Fecha"] = tiempos

In [9]:
solucion_con["R_t"] = pd.Series([rep(ti) for ti in sol_con_interv.t],index=sol_con_interv.t)

In [10]:
solucion_sin[solucion_sin["Fecha"]>"2020-04-01"]

,Susceptibles,Expuestos,Infectados,Leves,Graves,Hospitalizados,ICUs,Recuperados,Defunciones,Fecha
14,2.192687e+07,40777.038529,13371.245243,12802.045827,1310.043894,880.233552,114.879001,3.847460e+03,31.894097,2020-04-02
15,2.190711e+07,51727.032899,16961.131891,16271.449378,1662.582166,1119.383271,147.561806,4.962367e+03,42.471225,2020-04-03
16,2.188214e+07,65493.066317,21564.312070,20651.496378,2105.453167,1424.285630,188.825600,6.381451e+03,56.051051,2020-04-04
17,2.185047e+07,82907.954216,27430.238970,26208.090063,2666.358529,1812.358088,241.146334,8.188692e+03,73.460560,2020-04-05
18,2.181026e+07,105058.351372,34826.005867,33279.820324,3382.153135,2303.657282,307.628523,1.048528e+04,95.700885,2020-04-06
...,...,...,...,...,...,...,...,...,...,...
195,5.395168e+05,2.782056,-16.080566,587.097457,5.974264,-5.901763,1.680581,2.075023e+07,709678.770529,2020-09-30
196,5.395175e+05,1.440237,-8.323521,540.177793,3.092472,-3.019413,1.041372,2.075027e+07,709678.932148,2020-10-01
197,5.395178e+05,0.874827,-5.055191,500.327946,1.878250,-1.810388,0.747436,2.075031e+07,709679.016785,2020-10-02
198,5.395177e+05,0.968861,-5.599332,466.704675,2.080372,-2.022551,0.746986,2.075034e+07,709679.035883,2020-10-03


In [11]:
p_sin = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX")

In [12]:
for label in labels:
    p_sin.line(x=solucion_sin["Fecha"],y=solucion_sin[label]/1000,color=colores[label],line_width=3,legend_label=label,line_alpha=0.5)

In [13]:
p_sin.xaxis.axis_label = 'Fecha'
p_sin.yaxis.axis_label = 'Miles de personas'
p_sin.legend.location = "center_right"

In [14]:
bpl.show(p_sin)

In [15]:
rp = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX",y_range=(0,4))

In [16]:
rp.line(x=solucion_con["Fecha"],y=solucion_con["R_t"],color="black",line_width=3,legend_label="R(t)",line_alpha=0.8)

GlyphRenderer(id='1612', ...)

In [17]:
rp.xaxis.axis_label = 'Fecha'
rp.yaxis.axis_label = 'R_t'
rp.legend.location = "center_right"

In [18]:
bpl.show(rp)

In [19]:
p_con = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX")

In [20]:
for label in ["ICUs"]:
    p_con.line(x=solucion_con["Fecha"],y=solucion_con[label]/1000,color=colores[label],line_width=3,legend_label=label,line_alpha=0.5,line_dash="dotted")

In [21]:
p_con.xaxis.axis_label = 'Fecha'
p_con.yaxis.axis_label = 'Miles de personas'
p_con.legend.location = "center_right"

In [22]:
bpl.show(p_con)

Definición de función para evaluar el modelo en función de R_0 y calculando el error a los datos reales. esto es la función que usaremos para ajustar R_0 a los datos. Se puede seguir una estrategia similar si se desean ajustar más parámetros. Dos parámetros son importantes, los pesos, el primero 't_pesos' indica si el error entre los datos y el modelo es pesado dando prioridad a los datos que estén más cercanos al último día de ajuste. El segundo indica los pesos para cada una de las series a ajustar, las series a ajustar hasta el momento son Confirmados acumulados, Defunciones acumuladas, Casos graves. Se consideran varias series para tratar de quitar dependencia a la tasa de detección de casos.

In [23]:
def evaluador(R_0, casos, N, dias_prediccion=0,t_pesos=True,pesos_ajuste_CDG=[1,1,1],regresar_solucion=False):
    inicio = casos.index[0]
    n_infected = casos["Confirmados"].fillna(0)[inicio]
    max_days = len(casos) + dias_prediccion
    tiempos = pd.date_range(start=inicio, periods=max_days).values
    serie_tiempos = pd.Series(tiempos)
    
    initial_state = [(N - n_infected)/ N, 0, n_infected / N, 0, 0, 0, 0, 0, 0]

    t_inf=2.9
    t_inc=5.2
    t_rh=12
    t_rl=14
    t_hosp=4
    t_icu=1
    t_ricu = 7
    t_m=8
    p_m=0.65
    p_grave=0.138
    p_icu=0.05
    
    args = (R_0, t_inf, t_inc, t_rh, t_rl, t_hosp, t_icu, t_ricu, t_m, p_m, p_grave, p_icu)
    
    sol = solve_ivp(CDMX_model, [0, max_days], initial_state, args=args, t_eval=np.arange(max_days))
    
    labels = ["Susceptibles","Expuestos","Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]
    
    solucion = pd.DataFrame(sol.y.T*N,columns=labels,index=tiempos)
    solucion["Defunciones"] = np.clip(solucion["Defunciones"],0,np.inf)
    solucion["Graves"] = np.clip(solucion["Graves"],0,np.inf)
    solucion["Confirmados_acumulados"] = np.clip(solucion[["Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]].sum(axis=1),0,np.inf)
    
    if t_pesos:
        pesos = pd.Series(1/((casos.index-casos.index[-1]).days*-1+1),index=casos.index)
    else:
        pesos = pd.Series(1,index=casos.index)
    
    error_confirmados = mean_squared_log_error(casos["Confirmados_acumulados"],solucion["Confirmados_acumulados"][casos.index].fillna(0),pesos)
    error_decesos = mean_squared_log_error(casos["Decesos_acumulados"],solucion["Defunciones"][casos.index].fillna(0),pesos)
    error_graves = mean_squared_log_error(casos["Graves"].fillna(0),solucion["Graves"][casos.index].fillna(0),pesos)
    
    errores = [error_confirmados,error_decesos,error_graves]
        
    error_final = np.dot(pesos_ajuste_CDG,errores)/len(errores)
    
    if regresar_solucion:
        return error_final, solucion
    else:
        return error_final

Obtención de los últimos datos para México

In [24]:
from herramientas import arbol
arbol(DATOS_BRUTOS)

+ ../datos/brutos
    + 200412COVID19MEXICO.csv
    + 200413COVID19MEXICO.csv
    + 200414COVID19MEXICO.csv
    + 200415COVID19MEXICO.csv
    + 200416COVID19MEXICO.csv


In [25]:
archivo = sorted(list(DATOS_BRUTOS.glob("*COVID19MEXICO.csv")))[-1]
fecha_archivo = dt.strptime(archivo.name[:6],"%y%m%d").date()

In [26]:
data = pd.read_csv(archivo, encoding="latin1", low_memory=False)

In [27]:
data["FECHA_ACTUALIZACION"] = pd.to_datetime(data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
data["FECHA_INGRESO"] = pd.to_datetime(data["FECHA_INGRESO"],format="%Y-%m-%d")
data["FECHA_DEF"] = pd.to_datetime(data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
data["FECHA_SINTOMAS"] = pd.to_datetime(data["FECHA_SINTOMAS"],format="%Y-%m-%d")

In [28]:
data[:3]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-04-19,09e8dc,2,9,15,1,15,15,37,2,...,2,2,2,2,2,1,99,99,97,1
1,2020-04-19,1dd782,2,12,9,1,15,9,3,1,...,2,1,2,2,2,3,99,99,97,97
2,2020-04-19,0efbaf,2,9,28,2,16,28,32,1,...,2,2,2,2,1,1,99,99,97,97


Filtrado de datos

In [29]:
sun_data_mun = pd.read_csv(DATOS_EXTERNOS / "Proy_ZM_2015_mun.csv", encoding="latin1")
sun_data = pd.read_csv(DATOS_EXTERNOS / "Proy_ZM_2015.csv", encoding="latin1")

In [30]:
sun_data_mun["CVE_MUN"] = sun_data_mun["CVE_MUN"].astype(str).str.zfill(5)

In [31]:
suns = pd.merge(sun_data[["CVE_SUN","NOM_SUN","P_2020"]],sun_data_mun.groupby("CVE_SUN")["CVE_MUN"].unique().to_frame("claves").reset_index(),how="outer",on="CVE_SUN")

In [32]:
suns.sort_values("P_2020",ascending=False)["NOM_SUN"].unique()

array(['Valle de México', 'Guadalajara', 'Monterrey', 'Puebla-Tlaxcala',
       'Toluca', 'Tijuana', 'León', 'Querétaro', 'Juárez', 'La Laguna',
       'Mérida', 'San Luis Potosí', 'Aguascalientes', 'Mexicali',
       'Cuernavaca', 'Saltillo', 'Chihuahua', 'Tampico', 'Morelia',
       'Culiacán', 'Veracruz', 'Hermosillo', 'Acapulco',
       'Tuxtla Gutiérrez', 'Villahermosa', 'Cancún', 'Reynosa', 'Xalapa',
       'Celaya', 'Oaxaca', 'Durango', 'Pachuca', 'Tlaxcala-Apizaco',
       'Poza Rica', 'Matamoros', 'Ensenada', 'Mazatlán', 'Cuautla',
       'Tepic', 'Orizaba', 'Puerto Vallarta', 'Nuevo Laredo',
       'Minatitlán', 'Colima-Villa de Álvarez', 'Zacatecas-Guadalupe',
       'Monclova-Frontera', 'Coatzacoalcos', 'Tapachula',
       'Ciudad Victoria', 'Tehuacán', 'Córdoba', 'Chilpancingo',
       'Campeche', 'La Paz', 'Zamora', 'Tulancingo', 'La Piedad-Pénjamo',
       'Chetumal', 'Nogales', 'Tula', 'Guaymas',
       'San Francisco del Rincón', 'Piedras Negras', 'Delicias',
       'G

In [33]:
zona_met = 'Valle de México'
z_urbana =suns[suns["NOM_SUN"]==zona_met]
poblacion = z_urbana["P_2020"].values[0]
z_urbana

,CVE_SUN,NOM_SUN,P_2020,claves
18,M09.01,Valle de México,21942666,"[09002, 09003, 09004, 09005, 09006, 09007, 090..."


In [34]:
data["CVE_MUN_RES"] = data["ENTIDAD_RES"].astype(str).str.zfill(2) + data["MUNICIPIO_RES"].astype(str).str.zfill(3)

In [35]:
data_zurb = data[(data["RESULTADO"]==1)&(data["CVE_MUN_RES"].isin(z_urbana["claves"].values[0]))]

In [37]:
data_zurb[:3]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,CVE_MUN_RES
0,2020-04-19,09e8dc,2,9,15,1,15,15,37,2,...,2,2,2,2,1,99,99,97,1,15037
6,2020-04-19,0cdb12,1,12,9,2,99,9,12,2,...,2,2,2,2,1,99,MÃ©xico,97,2,09012
7,2020-04-19,038dad,1,12,9,1,99,9,14,1,...,2,2,2,1,1,99,MÃ©xico,97,97,09014


In [38]:
time_index = pd.date_range(data_zurb["FECHA_SINTOMAS"].min(), fecha_archivo, freq='1D')

In [39]:
time_index

DatetimeIndex(['2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20',
               '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24',
               '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28',
               '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03',
               '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07',
               '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11',
               '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15',
               '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19',
               '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23',
               '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27',
               '2020-03-28', '2020-03-29', '2020-03-30', '2020-03-31',
               '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04',
               '2020-04-05', '2020-04-06', '2020-04-07', '2020-04-08',
               '2020-04-09', '2020-04-10', '2020-04-11', '2020-04-12',
      

In [40]:
casos = data_zurb.groupby("FECHA_SINTOMAS").size().to_frame("Confirmados").reindex(time_index)

In [41]:
casos["Graves"] = data_zurb[data_zurb["TIPO_PACIENTE"]==2].groupby(["FECHA_SINTOMAS"]).size()

In [42]:
casos["Decesos"] = data_zurb.groupby("FECHA_DEF").size()

In [43]:
casos["Confirmados_acumulados"] = casos["Confirmados"].fillna(0).cumsum()
casos["Decesos_acumulados"] = casos["Decesos"].fillna(0).cumsum()
casos["Graves_acumulados"] = casos["Graves"].fillna(0).cumsum()

In [44]:
casos[-3:]

,Confirmados,Graves,Decesos,Confirmados_acumulados,Decesos_acumulados,Graves_acumulados
2020-04-14,7.0,2.0,6.0,2387.0,148.0,946.0
2020-04-15,1.0,NaN,4.0,2388.0,152.0,946.0
2020-04-16,NaN,NaN,NaN,2388.0,152.0,946.0


Graficado de datos para Zona Urbana

In [45]:
inifecha = casos.index.min()
finfecha = casos.index.max()

In [46]:
maxcas = casos["Confirmados_acumulados"][finfecha]
maxcasd = casos["Confirmados"][inifecha:finfecha].max()

In [47]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=600,x_range=(inifecha,finfecha),y_range=(0,maxcas))

In [48]:
s.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Confirmados_acumulados"],bottom=0)
s.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Graves_acumulados"],bottom=0,color="green")
s.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Decesos_acumulados"],bottom=0,color="red")

GlyphRenderer(id='2110', ...)

In [49]:
bpl.show(s)

In [50]:
s0 = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=600,x_range=(inifecha,finfecha),y_range=(0,maxcasd))

In [51]:
s0.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Confirmados"],bottom=0)
s0.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Graves"],bottom=0,color="green")
s0.quad(left=time_index,right=time_index+pd.Timedelta("1 days")*0.9,top=casos["Decesos"],bottom=0,color="red")

GlyphRenderer(id='2409', ...)

In [52]:
bpl.show(s0)

Ejecución del modelo con la R_0 que reportan en CDMX

In [53]:
fecini = pd.Timestamp("2020-02-17")
fecfin = pd.Timestamp("2020-03-22")
dias_pred = 0
fecpred = fecfin + pd.Timedelta(str(dias_pred) + " days")

In [54]:
casos_filt = casos.loc[fecini:fecfin,:]
casos_pred = casos.loc[fecfin:fecpred,:]

In [55]:
casos_filt[:3]

,Confirmados,Graves,Decesos,Confirmados_acumulados,Decesos_acumulados,Graves_acumulados
2020-02-17,2.0,NaN,NaN,2.0,0.0,0.0
2020-02-18,NaN,NaN,NaN,2.0,0.0,0.0
2020-02-19,1.0,NaN,NaN,3.0,0.0,0.0


In [56]:
R_0a = 2.83
error, solucion = evaluador(R_0a,casos_filt,poblacion,dias_pred,True,[1,1,1],True)

/Users/ssanchez/opt/anaconda3/envs/covid/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass sample_weight=2020-02-17    0.028571
2020-02-18    0.029412
2020-02-19    0.030303
2020-02-20    0.031250
2020-02-21    0.032258
2020-02-22    0.033333
2020-02-23    0.034483
2020-02-24    0.035714
2020-02-25    0.037037
2020-02-26    0.038462
2020-02-27    0.040000
2020-02-28    0.041667
2020-02-29    0.043478
2020-03-01    0.045455
2020-03-02    0.047619
2020-03-03    0.050000
2020-03-04    0.052632
2020-03-05    0.055556
2020-03-06    0.058824
2020-03-07    0.062500
2020-03-08    0.066667
2020-03-09    0.071429
2020-03-10    0.076923
2020-03-11    0.083333
2020-03-12    0.090909
2020-03-13    0.100000
2020-03-14    0.111111
2020-03-15    0.125000
2020-03-16    0.142857
2020-03-17    0.166667
2020-03-18    0.200000
2020-03-19    0.250000
2020-03-20    0.333333
2020-03-21    0.500000
2020-03-22    1.000000
Freq: D, dtype: float64 as keyword args. From version 0.25 p

In [57]:
solucion[-10:]

,Susceptibles,Expuestos,Infectados,Leves,Graves,Hospitalizados,ICUs,Recuperados,Defunciones,Confirmados_acumulados
2020-03-13,2.194231e+07,179.075059,50.110723,77.771786,7.772819,5.185881,0.962900,32.049984,0.404055,174.258148
2020-03-14,2.194227e+07,181.204374,76.041241,84.590285,7.505090,6.897957,1.079797,38.915291,0.492650,215.522311
2020-03-15,2.194221e+07,192.490593,102.191446,94.844858,7.668123,8.722747,1.236427,46.870721,0.595861,262.130183
2020-03-16,2.194210e+07,261.446962,103.840948,120.726795,10.912067,9.571844,1.536261,55.070394,0.705367,302.363676
2020-03-17,2.194197e+07,342.042243,106.451005,151.134994,14.693893,10.608610,1.889437,64.829813,0.835692,350.443444
2020-03-18,2.194185e+07,395.904390,131.640176,176.906544,16.888034,12.828494,2.222677,77.346243,1.000242,418.832409
2020-03-19,2.194173e+07,429.655079,177.505644,200.047446,17.835298,16.178774,2.556202,92.880512,1.202744,508.206618
2020-03-20,2.194156e+07,493.806729,219.209310,233.461065,20.282284,19.584866,3.001484,110.782401,1.437515,607.758925
2020-03-21,2.194134e+07,614.540935,245.105262,284.121160,25.635182,22.569795,3.621396,131.029051,1.705672,713.787518
2020-03-22,2.194108e+07,741.637402,284.398179,340.247702,31.099211,26.495723,4.323806,155.420542,2.027777,844.012941


Graficado de la solución, se puede apreciar que el valor de R_0 no refleja bien el crecimiento que tienen los datos reales

In [58]:
p = bpl.figure(x_axis_type="datetime",plot_width=800,plot_height=600,title="Modelo CDMX, Zona Metropolitana de " + zona_met,y_axis_type="log")

In [59]:
p.line(x=casos_filt.index,y=casos_filt["Confirmados_acumulados"],color="#0B9A53",line_width=3,legend_label="Casos a ajustar",line_dash="dashed")
p.line(x=casos_pred.index,y=casos_pred["Confirmados_acumulados"],color="#3999E4",line_width=3,legend_label="Casos a predecir",line_dash="dotted")
p.line(x=solucion.index,y=solucion["Confirmados_acumulados"],color="#9B0E63",line_width=3,legend_label="Casos confirmados, modelo R_0 = " + str(R_0a))
for label in ["Infectados","Leves","Graves","Hospitalizados","ICUs","Recuperados","Defunciones"]:
    p.line(x=solucion.index,y=solucion[label],color=colores[label],line_width=3,legend_label=label)

In [60]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos'
p.legend.location = "bottom_right"

In [61]:
bpl.show(p)

Ajuste del modelo para obtener la mejor R_0

In [62]:
pesos_ajuste_CDG = [1,1,1]
ajuste = minimize(evaluador,[2.8],args=(casos_filt,poblacion,0,True,pesos_ajuste_CDG,False),method='L-BFGS-B')
ajuste

/Users/ssanchez/opt/anaconda3/envs/covid/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass sample_weight=2020-02-17    0.028571
2020-02-18    0.029412
2020-02-19    0.030303
2020-02-20    0.031250
2020-02-21    0.032258
2020-02-22    0.033333
2020-02-23    0.034483
2020-02-24    0.035714
2020-02-25    0.037037
2020-02-26    0.038462
2020-02-27    0.040000
2020-02-28    0.041667
2020-02-29    0.043478
2020-03-01    0.045455
2020-03-02    0.047619
2020-03-03    0.050000
2020-03-04    0.052632
2020-03-05    0.055556
2020-03-06    0.058824
2020-03-07    0.062500
2020-03-08    0.066667
2020-03-09    0.071429
2020-03-10    0.076923
2020-03-11    0.083333
2020-03-12    0.090909
2020-03-13    0.100000
2020-03-14    0.111111
2020-03-15    0.125000
2020-03-16    0.142857
2020-03-17    0.166667
2020-03-18    0.200000
2020-03-19    0.250000
2020-03-20    0.333333
2020-03-21    0.500000
2020-03-22    1.000000
Freq: D, dtype: float64 as keyword args. From version 0.25 p

      fun: 0.11383316697824224
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([3.60822483e-08])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 10
      nit: 3
   status: 0
  success: True
        x: array([2.71102017])

In [63]:
R_f = ajuste.x[0]
R_f

2.711020167650491

In [64]:
error, solucion = evaluador(R_f,casos_filt,poblacion,0,True,pesos_ajuste_CDG,True)

/Users/ssanchez/opt/anaconda3/envs/covid/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass sample_weight=2020-02-17    0.028571
2020-02-18    0.029412
2020-02-19    0.030303
2020-02-20    0.031250
2020-02-21    0.032258
2020-02-22    0.033333
2020-02-23    0.034483
2020-02-24    0.035714
2020-02-25    0.037037
2020-02-26    0.038462
2020-02-27    0.040000
2020-02-28    0.041667
2020-02-29    0.043478
2020-03-01    0.045455
2020-03-02    0.047619
2020-03-03    0.050000
2020-03-04    0.052632
2020-03-05    0.055556
2020-03-06    0.058824
2020-03-07    0.062500
2020-03-08    0.066667
2020-03-09    0.071429
2020-03-10    0.076923
2020-03-11    0.083333
2020-03-12    0.090909
2020-03-13    0.100000
2020-03-14    0.111111
2020-03-15    0.125000
2020-03-16    0.142857
2020-03-17    0.166667
2020-03-18    0.200000
2020-03-19    0.250000
2020-03-20    0.333333
2020-03-21    0.500000
2020-03-22    1.000000
Freq: D, dtype: float64 as keyword args. From version 0.25 p

In [65]:
error

0.11383316697824224

In [66]:
solucion[-3:]

,Susceptibles,Expuestos,Infectados,Leves,Graves,Hospitalizados,ICUs,Recuperados,Defunciones,Confirmados_acumulados
2020-03-20,2.194184e+07,353.145060,167.920343,178.652739,14.957204,15.576802,2.411683,90.555266,1.222535,471.296572
2020-03-21,2.194168e+07,434.856114,186.914964,215.142432,18.712425,17.829545,2.880384,106.233228,1.439171,549.152149
2020-03-22,2.194150e+07,527.898166,210.910578,257.271635,22.953359,20.555808,3.424774,124.744414,1.694788,641.555355


Graficación del modelo ajustado

In [67]:
p_casos = bpl.figure(x_axis_type="datetime",plot_width=600,plot_height=400,title="Modelo CDMX, Zona Metropolitana de " + zona_met)
p_def = bpl.figure(x_axis_type="datetime",plot_width=600,plot_height=400)
p_graves = bpl.figure(x_axis_type="datetime",plot_width=600,plot_height=400)

In [68]:
p_casos.line(x=casos_filt.index,y=casos_filt["Confirmados_acumulados"],color="#0B9A53",line_width=3,legend_label="Casos",line_dash="dashed")
p_def.line(x=casos_filt.index,y=casos_filt["Decesos_acumulados"],color="#0B9A53",line_width=3,legend_label="Defunciones",line_dash="dashed")
p_graves.line(x=casos_filt.index,y=casos_filt["Graves"],color="#0B9A53",line_width=3,legend_label="Casos graves",line_dash="dashed")
for label in ["Confirmados_acumulados"]:
    p_casos.line(x=solucion.index,y=solucion[label],color="#9B0E63",line_width=3,legend_label="modelo R_0 = " + str(R_f))
for label in ["Defunciones"]:
    p_def.line(x=solucion.index,y=solucion[label],color="#9B0E63",line_width=3,legend_label="modelo R_0 = " + str(R_f))
for label in ["Graves"]:
    p_graves.line(x=solucion.index,y=solucion[label],color="#9B0E63",line_width=3,legend_label="modelo R_0 = " + str(R_f))

In [69]:
p_casos.xaxis.axis_label = 'Fecha'
p_casos.yaxis.axis_label = 'Casos'
p_casos.legend.location = "top_left"
p_def.xaxis.axis_label = 'Fecha'
p_def.yaxis.axis_label = 'Defunciones'
p_def.legend.location = "top_left"
p_graves.xaxis.axis_label = 'Fecha'
p_graves.yaxis.axis_label = 'Casos Graves'
p_graves.legend.location = "top_left"

In [70]:
bpl.show(bly.column(p_casos,p_def,p_graves))